# 08 - MLlib ALS Deep Dive

Pogłębiona nauka algorytmu ALS (Alternating Least Squares) z Spark MLlib.

**Tematy:**
- Jak działa ALS - teoria
- Hyperparameter tuning z CrossValidator i ParamGridBuilder
- Train / Validation / Test split
- Strategie cold start
- Metryki ewaluacji: RMSE, MAE, NDCG
- Zapis i odczyt modelu (persistence)
- Item-item similarity z wektorów latentnych
- Implicit feedback ALS

## 1. Setup

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder \
    .appName("08_MLlib_ALS") \
    .master("spark://spark-master:7077") \
    .config("spark.jars.packages", "org.postgresql:postgresql:42.7.1") \
    .config("spark.driver.memory", "6g") \
    .config("spark.executor.memory", "7g") \
    .config("spark.driver.host", "recommender-jupyter") \
    .config("spark.driver.bindAddress", "0.0.0.0") \
    .getOrCreate()

jdbc_url = "jdbc:postgresql://postgres:5432/recommender"
properties = {
    "user": "recommender",
    "password": "recommender",
    "driver": "org.postgresql.Driver"
}

ratings = spark.read.jdbc(
    jdbc_url, "movielens.ratings", properties=properties,
    column="user_id", lowerBound=1, upperBound=300000, numPartitions=10
)
movies = spark.read.jdbc(jdbc_url, "movielens.movies", properties=properties)

ratings.cache()
print(f"Ratings: {ratings.count()}, Movies: {movies.count()}")

:: loading settings :: url = jar:file:/app/.venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /root/.ivy2.5.2/cache
The jars for the packages stored in: /root/.ivy2.5.2/jars
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9060aa95-6045-4039-883d-07ff322ee4f5;1.0
	confs: [default]
	found org.postgresql#postgresql;42.7.1 in central
	found org.checkerframework#checker-qual;3.41.0 in central
:: resolution report :: resolve 96ms :: artifacts dl 3ms
	:: modules in use:
	org.checkerframework#checker-qual;3.41.0 from central in [default]
	org.postgresql#postgresql;42.7.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	-----------------------------------------

Ratings: 20000263, Movies: 27278


## 2. Jak działa ALS - teoria

ALS (Alternating Least Squares) to algorytm **collaborative filtering** oparty o **matrix factorization**.

### Idea:
Macierz ocen R (users × items) jest rzadka (sparse). ALS rozkłada ją na dwie mniejsze macierze:

```
R ≈ U × I^T
```

- **U** (users × rank) - wektor latentny użytkownika
- **I** (items × rank) - wektor latentny filmu
- **rank** - liczba latentnych czynników (hyperparameter)

### Algorytm:
1. Zainicjalizuj U i I losowo
2. Zamroź I, optymalizuj U (least squares)
3. Zamroź U, optymalizuj I (least squares)
4. Powtórz 2-3 przez maxIter iteracji

### Kluczowe parametry:
- **rank** - wymiar wektora latentnego (10-200)
- **maxIter** - liczba iteracji (5-20)
- **regParam** - regularyzacja L2 (0.01-1.0)
- **alpha** - parametr confidence (tylko implicit feedback)
- **implicitPrefs** - czy używać implicit feedback

## 3. Train / Validation / Test Split

Prawidłowy podział danych:
- **Training** (60%) - model się uczy
- **Validation** (20%) - tuning hyperparametrów
- **Test** (20%) - finalna ewaluacja (użyj tylko RAZ!)

In [ ]:
# Podział danych
(training, validation, test) = ratings.randomSplit([0.6, 0.2, 0.2], seed=42)

# Cache bo będziemy wielokrotnie używać
training.cache()
validation.cache()
test.cache()

print(f"Training: {training.count()}")
print(f"Validation: {validation.count()}")
print(f"Test: {test.count()}")

## 4. Baseline - prosty model ALS

In [ ]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

# Baseline model
als_baseline = ALS(
    maxIter=10,
    regParam=0.1,
    rank=10,
    userCol="user_id",
    itemCol="movie_id",
    ratingCol="rating",
    coldStartStrategy="drop",
    seed=42
)

model_baseline = als_baseline.fit(training)

# Ewaluacja na validation
predictions = model_baseline.transform(validation)

evaluator_rmse = RegressionEvaluator(
    metricName="rmse", labelCol="rating", predictionCol="prediction"
)
evaluator_mae = RegressionEvaluator(
    metricName="mae", labelCol="rating", predictionCol="prediction"
)

rmse = evaluator_rmse.evaluate(predictions)
mae = evaluator_mae.evaluate(predictions)
print(f"Baseline RMSE: {rmse:.4f}")
print(f"Baseline MAE: {mae:.4f}")

## 5. Ręczny hyperparameter tuning

Zanim użyjemy CrossValidator, zróbmy to ręcznie żeby zrozumieć proces.

In [ ]:
import itertools

# Grid search - ręczny
ranks = [5, 10, 20]
reg_params = [0.01, 0.1, 0.5]

results = []

for rank, reg in itertools.product(ranks, reg_params):
    als = ALS(
        maxIter=10,
        regParam=reg,
        rank=rank,
        userCol="user_id",
        itemCol="movie_id",
        ratingCol="rating",
        coldStartStrategy="drop",
        seed=42
    )
    model = als.fit(training)
    preds = model.transform(validation)
    rmse = evaluator_rmse.evaluate(preds)
    mae = evaluator_mae.evaluate(preds)
    results.append((rank, reg, rmse, mae))
    print(f"rank={rank:2d}, regParam={reg:.2f} → RMSE={rmse:.4f}, MAE={mae:.4f}")

# Najlepszy model
best = min(results, key=lambda x: x[2])
print(f"\nNajlepszy: rank={best[0]}, regParam={best[1]}, RMSE={best[2]:.4f}")

## 6. CrossValidator - automatyczny hyperparameter tuning

CrossValidator automatycznie:
1. Tworzy kombinacje hyperparametrów (ParamGrid)
2. Dzieli dane na K foldów
3. Trenuje model na K-1 foldach, ewaluuje na 1 foldzie
4. Powtarza K razy
5. Uśrednia metryki i wybiera najlepszą kombinację

In [ ]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

als = ALS(
    userCol="user_id",
    itemCol="movie_id",
    ratingCol="rating",
    coldStartStrategy="drop",
    seed=42
)

# ParamGrid - siatka hyperparametrów do przeszukania
param_grid = ParamGridBuilder() \
    .addGrid(als.rank, [10, 20, 50]) \
    .addGrid(als.maxIter, [10, 15]) \
    .addGrid(als.regParam, [0.01, 0.1, 0.3]) \
    .build()

print(f"Liczba kombinacji do przetestowania: {len(param_grid)}")

# CrossValidator
cv = CrossValidator(
    estimator=als,
    estimatorParamMaps=param_grid,
    evaluator=evaluator_rmse,
    numFolds=3,  # 3-fold CV
    seed=42
)

# Uwaga: to jest BARDZO kosztowne obliczeniowo!
# 18 kombinacji × 3 foldy = 54 modele do wytrenowania
# Na dużych danych może trwać godzinami

In [ ]:
# Trenuj CrossValidator (to zajmie chwilę!)
# Użyj mniejszej próbki jeśli trwa zbyt długo
training_sample = training.sample(0.1, seed=42).cache()
print(f"Training sample: {training_sample.count()} rows")

cv_model = cv.fit(training_sample)

In [ ]:
# Wyniki Cross Validation
print("Średnie RMSE per kombinacja:")
for params, metric in zip(param_grid, cv_model.avgMetrics):
    rank = params[als.rank]
    maxIter = params[als.maxIter]
    regParam = params[als.regParam]
    print(f"  rank={rank:2d}, maxIter={maxIter:2d}, regParam={regParam:.2f} → RMSE={metric:.4f}")

# Najlepszy model
best_model = cv_model.bestModel
print(f"\nNajlepszy model:")
print(f"  rank: {best_model.rank}")
print(f"  Validation RMSE: {min(cv_model.avgMetrics):.4f}")

### TrainValidationSplit - szybsza alternatywa

Zamiast K-fold CV, trenuje na jednym splicie. Szybszy ale mniej dokładny.

In [ ]:
from pyspark.ml.tuning import TrainValidationSplit

tvs = TrainValidationSplit(
    estimator=als,
    estimatorParamMaps=param_grid,
    evaluator=evaluator_rmse,
    trainRatio=0.8,  # 80% train, 20% validation
    seed=42
)

# Szybszy niż CrossValidator!
tvs_model = tvs.fit(training_sample)

best_tvs_model = tvs_model.bestModel
print(f"Najlepszy model (TVS): rank={best_tvs_model.rank}")
print(f"Best RMSE: {min(tvs_model.validationMetrics):.4f}")

### Zadanie 1
Wytrenuj najlepszy model na pełnych danych treningowych z najlepszymi hyperparametrami znalezionymi powyżej.
Ewaluuj na zbiorze walidacyjnym.

In [ ]:
# Twoje rozwiązanie:


## 7. Strategie Cold Start

**Cold start problem** - nowy użytkownik/film bez ocen. Model nie ma informacji.

ALS coldStartStrategy:
- `"nan"` - zwraca NaN (domyślne)
- `"drop"` - pomija wiersze z NaN w predykcji

Dodatkowe strategie (do implementacji ręcznej):
- Popularne filmy (fallback)
- Content-based features
- Hybrydowe podejścia

In [ ]:
# Wytrenuj model na zbiorze treningowym
als_full = ALS(
    maxIter=10, regParam=0.1, rank=20,
    userCol="user_id", itemCol="movie_id", ratingCol="rating",
    coldStartStrategy="nan",  # zostawmy NaN żeby zobaczyć problem
    seed=42
)
model_full = als_full.fit(training)

# Predykcja na test - mogą być NaN
preds_nan = model_full.transform(test)
nan_count = preds_nan.filter(col("prediction").isNull() | isnan(col("prediction"))).count()
total = preds_nan.count()
print(f"NaN predictions: {nan_count} / {total} ({nan_count/total*100:.1f}%)")

In [ ]:
# Strategia fallback: zastąp NaN popularnymi filmami
global_avg = training.agg(avg("rating")).collect()[0][0]
print(f"Globalna średnia ocena: {global_avg:.2f}")

# Zastąp NaN globalną średnią
preds_filled = preds_nan.withColumn(
    "prediction_filled",
    when(col("prediction").isNull() | isnan(col("prediction")), global_avg)
    .otherwise(col("prediction"))
)

# Ewaluuj z fallback
evaluator_filled = RegressionEvaluator(
    metricName="rmse", labelCol="rating", predictionCol="prediction_filled"
)
rmse_filled = evaluator_filled.evaluate(preds_filled)
print(f"RMSE z fallback: {rmse_filled:.4f}")

In [ ]:
# Lepszy fallback: średnia ocena per film (zamiast globalnej)
movie_avg = training.groupBy("movie_id").agg(
    avg("rating").alias("movie_avg_rating")
)

preds_smart = preds_nan.join(movie_avg, "movie_id", "left") \
    .withColumn(
        "prediction_smart",
        when(col("prediction").isNull() | isnan(col("prediction")),
             coalesce(col("movie_avg_rating"), lit(global_avg)))
        .otherwise(col("prediction"))
    )

evaluator_smart = RegressionEvaluator(
    metricName="rmse", labelCol="rating", predictionCol="prediction_smart"
)
rmse_smart = evaluator_smart.evaluate(preds_smart)
print(f"RMSE z smart fallback: {rmse_smart:.4f}")

## 8. Metryki ewaluacji

### Rating prediction metrics:
- **RMSE** - Root Mean Squared Error (karze duże błędy)
- **MAE** - Mean Absolute Error (bardziej odporny na outliers)

### Ranking metrics (ważniejsze dla rekomendacji!):
- **Precision@K** - ile z top K rekomendacji jest trafnych
- **Recall@K** - ile trafnych filmów jest w top K
- **NDCG@K** - Normalized Discounted Cumulative Gain (uwzględnia pozycję)

In [ ]:
# Model z coldStartStrategy="drop" do metryk rankingowych
als_ranking = ALS(
    maxIter=10, regParam=0.1, rank=20,
    userCol="user_id", itemCol="movie_id", ratingCol="rating",
    coldStartStrategy="drop", seed=42
)
model_ranking = als_ranking.fit(training)

# RMSE i MAE
preds = model_ranking.transform(test)
print(f"RMSE: {evaluator_rmse.evaluate(preds):.4f}")
print(f"MAE:  {evaluator_mae.evaluate(preds):.4f}")

In [ ]:
# Precision@K i Recall@K - ręczna implementacja
K = 10
THRESHOLD = 4.0  # filmy z oceną >= 4.0 uznajemy za "trafne"

# Ground truth: filmy które użytkownik ocenił wysoko w zbiorze testowym
relevant_items = test.filter(col("rating") >= THRESHOLD) \
    .groupBy("user_id") \
    .agg(collect_set("movie_id").alias("relevant_movies"))

# Top K rekomendacji z modelu
user_recs = model_ranking.recommendForAllUsers(K)

# Rozpakuj rekomendacje
from pyspark.sql.functions import col, expr

user_recs_flat = user_recs.select(
    col("user_id"),
    expr("transform(recommendations, x -> x.movie_id)").alias("recommended_movies")
)

user_recs_flat.show(5, truncate=False)

In [ ]:
# Join i policz Precision@K, Recall@K
metrics = user_recs_flat.join(relevant_items, "user_id") \
    .withColumn(
        "hits",
        size(array_intersect(col("recommended_movies"), col("relevant_movies")))
    ) \
    .withColumn("precision_at_k", col("hits") / K) \
    .withColumn("recall_at_k", col("hits") / size(col("relevant_movies")))

avg_metrics = metrics.agg(
    round(avg("precision_at_k"), 4).alias("avg_precision_at_k"),
    round(avg("recall_at_k"), 4).alias("avg_recall_at_k")
)

avg_metrics.show()

# Rozkład metryk
metrics.select("precision_at_k", "recall_at_k").summary().show()

In [ ]:
# NDCG@K - Normalized Discounted Cumulative Gain
# Uwzględnia POZYCJĘ trafienia - trafienie na pozycji 1 jest cenniejsze niż na pozycji 10

from pyspark.sql.types import FloatType, ArrayType
import numpy as np

@udf(FloatType())
def ndcg_at_k(recommended, relevant, k=10):
    """Calculate NDCG@K."""
    if not recommended or not relevant:
        return 0.0
    
    relevant_set = set(relevant)
    
    # DCG
    dcg = 0.0
    for i, item in enumerate(recommended[:k]):
        if item in relevant_set:
            dcg += 1.0 / np.log2(i + 2)  # +2 bo log2(1)=0
    
    # Ideal DCG (wszystkie trafienia na początku)
    ideal_hits = min(len(relevant_set), k)
    idcg = sum(1.0 / np.log2(i + 2) for i in range(ideal_hits))
    
    return float(dcg / idcg) if idcg > 0 else 0.0

# Policz NDCG@10
ndcg_result = user_recs_flat.join(relevant_items, "user_id") \
    .withColumn("ndcg", ndcg_at_k(col("recommended_movies"), col("relevant_movies")))

avg_ndcg = ndcg_result.agg(round(avg("ndcg"), 4).alias("avg_ndcg_at_10")).collect()[0][0]
print(f"Average NDCG@10: {avg_ndcg}")

### Zadanie 2
Porównaj metryki rankingowe (Precision@K, NDCG@K) dla modeli z rank=10 i rank=50.
Czy większy rank zawsze jest lepszy?

In [ ]:
# Twoje rozwiązanie:


## 9. Model Persistence - zapis i odczyt modelu

In [ ]:
# Zapisz model
model_path = "/tmp/als_model"
model_ranking.write().overwrite().save(model_path)
print(f"Model zapisany do: {model_path}")

In [ ]:
# Odczytaj model
from pyspark.ml.recommendation import ALSModel

loaded_model = ALSModel.load(model_path)
print(f"Załadowany model: rank={loaded_model.rank}")

# Weryfikacja - te same predykcje?
preds_loaded = loaded_model.transform(test)
rmse_loaded = evaluator_rmse.evaluate(preds_loaded)
print(f"RMSE załadowanego modelu: {rmse_loaded:.4f}")

In [ ]:
# Zapisz cały pipeline (CrossValidator model)
# cv_model.write().overwrite().save("/tmp/als_cv_model")

# Zapisz też metadane modelu
model_metadata = spark.createDataFrame([
    ("rank", str(model_ranking.rank)),
    ("rmse", str(rmse_loaded)),
    ("training_size", str(training.count())),
    ("timestamp", str(current_timestamp()))
], ["key", "value"])

model_metadata.show(truncate=False)

## 10. Item-Item Similarity z wektorów latentnych

Wektory latentne z ALS mogą być użyte do znalezienia podobnych filmów.

In [ ]:
# Pobierz item factors
item_factors = model_ranking.itemFactors
user_factors = model_ranking.userFactors

print(f"Item factors: {item_factors.count()} items × rank={model_ranking.rank}")
print(f"User factors: {user_factors.count()} users × rank={model_ranking.rank}")

item_factors.show(5, truncate=False)

In [ ]:
# Cosine similarity - znajdź filmy podobne do danego
import numpy as np
from pyspark.sql.types import DoubleType

def find_similar_movies(target_movie_id, n=10):
    """Znajdź n filmów najbardziej podobnych do target_movie_id."""
    
    # Pobierz wektor docelowego filmu
    target_vector = item_factors.filter(col("id") == target_movie_id) \
        .select("features").collect()[0][0]
    target_np = np.array(target_vector)
    
    # UDF do cosine similarity
    @udf(DoubleType())
    def cosine_sim(features):
        v = np.array(features)
        return float(np.dot(target_np, v) / (np.linalg.norm(target_np) * np.linalg.norm(v)))
    
    # Policz similarity dla wszystkich filmów
    similar = item_factors \
        .withColumn("similarity", cosine_sim(col("features"))) \
        .filter(col("id") != target_movie_id) \
        .orderBy(desc("similarity")) \
        .limit(n) \
        .withColumnRenamed("id", "movie_id") \
        .join(movies, "movie_id") \
        .select("movie_id", "title", "genres", "similarity")
    
    return similar

# Filmy podobne do Toy Story (1995) - movie_id=1
print("Filmy podobne do Toy Story (1995):")
find_similar_movies(1).show(truncate=False)

In [ ]:
# Filmy podobne do The Matrix (1999) - movie_id=2571
print("Filmy podobne do The Matrix (1999):")
find_similar_movies(2571).show(truncate=False)

In [ ]:
# Filmy podobne do The Shawshank Redemption (1994) - movie_id=318
print("Filmy podobne do The Shawshank Redemption (1994):")
find_similar_movies(318).show(truncate=False)

## 11. Implicit Feedback ALS

W wielu systemach nie mamy explicit ocen - mamy tylko sygnały implicit:
- Ile razy użytkownik obejrzał film
- Czy kliknął
- Czas spędzony na stronie

ALS z `implicitPrefs=True` interpretuje dane jako **confidence** a nie ocenę.

In [ ]:
# Symuluj implicit feedback - zamieniamy oceny na binarne "interakcje"
# rating >= 4.0 → 1 (user liked), reszta → 0
implicit_data = ratings.withColumn(
    "interaction",
    when(col("rating") >= 4.0, 1.0).otherwise(0.0)
)

implicit_data.groupBy("interaction").count().show()

(train_impl, test_impl) = implicit_data.randomSplit([0.8, 0.2], seed=42)

In [ ]:
# ALS z implicit feedback
als_implicit = ALS(
    maxIter=10,
    regParam=0.1,
    rank=20,
    userCol="user_id",
    itemCol="movie_id",
    ratingCol="interaction",
    implicitPrefs=True,  # implicit mode!
    alpha=40.0,  # confidence scaling factor
    coldStartStrategy="drop",
    seed=42
)

model_implicit = als_implicit.fit(train_impl)

# Rekomendacje - wartości w (0, 1) zamiast (0.5, 5.0)
user_42_recs = model_implicit.recommendForUserSubset(
    spark.createDataFrame([(42,)], ["user_id"]), 10
)

user_42_recs.select("user_id", explode("recommendations").alias("rec")) \
    .select("user_id", "rec.movie_id", "rec.rating") \
    .join(movies, "movie_id") \
    .select("title", "rating") \
    .show(truncate=False)

## Zadanie końcowe

Zbuduj pełny pipeline rekomendacji:

1. Podziel dane na train/validation/test (60/20/20)
2. Użyj TrainValidationSplit z ParamGrid (rank: [10, 30, 50], regParam: [0.05, 0.1, 0.2])
3. Wytrenuj najlepszy model na pełnym zbiorze treningowym
4. Ewaluuj na teście: RMSE, MAE, Precision@10, NDCG@10
5. Dla wybranego użytkownika pokaż:
   - Jego 10 najwyżej ocenionych filmów (ground truth)
   - 10 rekomendacji modelu
   - Czy rekomendacje mają sens?
6. Zapisz model do pliku

In [ ]:
# Twoje rozwiązanie:


In [ ]:
ratings.unpersist()
training.unpersist()
validation.unpersist()
test.unpersist()
spark.stop()